In [1]:
import os
# Find the latest version of spark 3.0 from http://www.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.0.3'
spark_version = 'spark-3.0.3'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop2.7.tgz
!tar xf $SPARK_VERSION-bin-hadoop2.7.tgz
!pip install -q findspark

# Set Environment Variables
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop2.7"

# Start a SparkSession
import findspark
findspark.init()

Get:1 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Ign:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Get:3 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Ign:4 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:5 http://archive.ubuntu.com/ubuntu bionic InRelease
Hit:6 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release
Get:7 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease [15.9 kB]
Hit:8 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Get:9 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Hit:10 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Get:12 http://archive.ubuntu.com/ubuntu bionic-backports InRelease [74.6 kB]
Hit:14 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease
Hit:15 http://ppa

In [2]:
# Download the Postgres driver that will allow Spark to interact with Postgres.
!wget https://jdbc.postgresql.org/download/postgresql-42.2.16.jar

--2021-08-18 22:44:20--  https://jdbc.postgresql.org/download/postgresql-42.2.16.jar
Resolving jdbc.postgresql.org (jdbc.postgresql.org)... 72.32.157.228, 2001:4800:3e1:1::228
Connecting to jdbc.postgresql.org (jdbc.postgresql.org)|72.32.157.228|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1002883 (979K) [application/java-archive]
Saving to: ‘postgresql-42.2.16.jar’

postgresql-42.2.16. 100%[===================>] 979.38K  1.21MB/s    in 0.8s    

2021-08-18 22:44:21 (1.21 MB/s) - ‘postgresql-42.2.16.jar’ saved [1002883/1002883]



In [4]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("Amazon-Reviews-Challenge").config("spark.driver.extraClassPath","/content/postgresql-42.2.16.jar").getOrCreate()

In [5]:
from pyspark import SparkFiles
url = "https://s3.amazonaws.com/amazon-reviews-pds/tsv/amazon_reviews_us_Books_v1_02.tsv.gz"
spark.sparkContext.addFile(url)
df = spark.read.option("encoding", "UTF-8").csv(SparkFiles.get(""), sep="\t", header=True, inferSchema=True)
df.show()

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|   12076615| RQ58W7SMO911M|0385730586|     122662979|Sisterhood of the...|           Books|          4|            2|          3|   N|                N|this book was a g...|this boook was a ...| 2005-10-14|
|         US|   12703090|  RF6IUKMGL8SF|0811828964|      56191234|The Bad Girl's Gu...|           Books|          3|    

In [ ]:
# Step 1: Filter the data and create a new DataFrame or table where the total_votes is equal to or greater than 20
vine_df = df.select(["review_id","star_rating","helpful_votes","total_votes","vine","verified_purchase"])
vine_filtered_votes_df = vine_df.filter("total_votes >= 20")
vine_filtered_votes_df.show()

In [ ]:
# Step 2: Filter vine_filtered_votes_df to retrieve all the rows where the number of helpful_votes divided by total_votes is equal to or greater than 50%
vine_helpful_votes_df = vine_filtered_votes_df.filter("(helpful_votes/total_votes)>=0.5")
vine_helpful_votes_df.show()

In [ ]:
# Step 3: Filter vine_helpful_votes_df and retrieve all the rows where a review was written as part of the Vine program (paid)
vine_paid_helpful_df = vine_helpful_votes_df.filter("vine == 'Y'")
vine_paid_helpful_df.show()

In [ ]:
# Step 4: Filter vine_helpful_votes_df and retrieve all the rows where a review was not written as part of the Vine program (unpaid)
vine_unpaid_helpful_df = vine_helpful_votes_df.filter("vine == 'N'")
vine_unpaid_helpful_df.show()

In [ ]:
# Determine the total number of helpful reviews
total_reviews = vine_helpful_votes_df.count()
total_reviews

In [ ]:
# Total paid reviews
total_paid_reviews_df = vine_helpful_votes_df.filter("vine =='Y'")
total_paid_reviews = total_paid_reviews_df.count()
total_paid_reviews

In [ ]:
# Total unpaid reviews
total_unpaid_reviews_df = vine_helpful_votes_df.filter("vine =='N'")
total_unpaid_reviews = total_unpaid_reviews_df.count()
total_unpaid_reviews

In [ ]:
# Step 5.2: Determine the number of 5-star reviews
total_five_star_reviews_df = vine_helpful_votes_df.filter("star_rating == 5")
total_five_star_reviews = total_five_star_reviews_df.count()
total_five_star_reviews

In [ ]:
# Determine the number of paid 5-star reviews 
total_paid_five_star_reviews_df = total_five_star_reviews_df.filter("vine =='Y'")
total_paid_five_star_reviews = total_paid_five_star_reviews_df.count()
total_paid_five_star_reviews

In [ ]:
# Determine the number of unpaid 5-star reviews 
total_unpaid_five_star_reviews_df = total_five_star_reviews_df.filter("vine =='N'")
total_unpaid_five_star_reviews = total_unpaid_five_star_reviews_df.count()
total_unpaid_five_star_reviews

In [ ]:
# Determine the percentage of unpaid over paid 5-star reviews 
unpaid_five_star_per_total_paid = (total_paid_five_star_reviews/total_paid_reviews)*100
unpaid_five_star_per_total_paid

In [ ]:
# Determine the percentage of unpaid over unpaid 5-star reviews 
unpaid_five_star_per_total_unpaid = (total_unpaid_five_star_reviews/total_unpaid_reviews)*100
unpaid_five_star_per_total_unpaid

In [ ]:
# Determine Paid reviews as percent of total 5-star reviews
paid_reviews_per_five_star_review = (total_paid_five_star_reviews/total_five_star_reviews)*100
paid_reviews_per_five_star_review

In [ ]:
# Determin unpaid reviews as percent of total 5-star reviews
unpaid_reviews_per_five_star_review = (total_unpaid_five_star_reviews/total_five_star_reviews)*100
unpaid_reviews_per_five_star_review